[Reference](https://levelup.gitconnected.com/building-a-secure-dashboard-application-with-flask-8c6a4c352932)

In [1]:
pip install flask flask-login flask-sqlalchemy waitress

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.3 MB/s eta 0:00:00


In [2]:
import logging
import os
import threading
import time
from logging.handlers import TimedRotatingFileHandler

from flask import Flask, render_template, request, url_for, redirect, flash
from flask_login import UserMixin, login_user, LoginManager, login_required, logout_user
from flask_sqlalchemy import SQLAlchemy
from waitress import serve
from werkzeug.security import check_password_hash, generate_password_hash

In [3]:
app = Flask(__name__)
app.config['SECRET_KEY'] = "python is the real deal !@#$%"
app.config['SQLALCHEMY_DATABASE_URI'] = "sqlite:///users.db"
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

class User(UserMixin, db.Model):
    __tablename__ = "users"
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(250), unique=True, nullable=False)
    password = db.Column(db.String(500), nullable=False)

with app.app_context():
    db.create_all()

login_manager = LoginManager()
login_manager.init_app(app)

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

@login_manager.unauthorized_handler
def unauthorized_callback():
    return redirect(url_for("login"))

@app.route('/login', methods=["GET", "POST"])
def login():
    login_form = forms.LoginForm()

    if login_form.validate_on_submit():
        username = login_form.username.data
        password = login_form.password.data

        user = User.query.filter_by(username=username).first()

        if not user:
            flash("That username does not exist, please try again.")
            return redirect(url_for('login'))
        elif not check_password_hash(user.password, password):
            flash('Password incorrect, please try again.')
            return redirect(url_for('login'))
        else:
            login_user(user)
            return redirect(url_for('home'))

    return render_template("login.html", form=login_form)

@app.route('/logout')
def logout():
    logout_user()
    return redirect(url_for('home'))

def user_creator(user, passwd):
    hash_and_salted_password = generate_password_hash(passwd, method='pbkdf2:sha256', salt_length=8)
    new_user = User(username=user, password=hash_and_salted_password)
    db.session.add(new_user)
    db.session.commit()

def password_changer(user, passwd):
    user = User.query.filter_by(username=user).first()
    hash_and_salted_password = generate_password_hash(passwd, method='pbkdf2:sha256', salt_length=8)
    user.password = hash_and_salted_password
    db.session.commit()

@app.route('/', methods=["GET", "POST"])
@login_required
def home():
    # Code for displaying graphs goes here
    return render_template("index.html", temp_html=temp_html_files, plot_ready=plot_ready,
                           reload_interval=config.DASHBOARD_PAGE_RELOAD_IN_SECONDS * 1000)

def background_task():
    while True:
        try:
            # Code for generating new values for graphs goes here
            # In my case I used Plotly to generate some graphs and save them
            # as html files. And them I render the html when login is successful
        except Exception as e:
            logging.info(f"{str(e)}")

        time.sleep(config.DASHBOARD_NEW_DATA_QUERY_INTERVAL_IN_SECONDS)

background_thread = threading.Thread(name='BackgroundThread', target=background_task)
background_thread.start()

log_format = config.log_format
log_file_name = f'{config.LOG_PATH}/simregdash.log'
log_handler = TimedRotatingFileHandler(log_file_name, when='midnight', backupCount=10)
logging.basicConfig(level=logging.INFO, format=log_format, handlers=[log_handler])
logging.info("App is initializing")

if __name__ == "__main__":
    serve(app, host="0.0.0.0", port=9090)